In [21]:
import pandas as pd
import os
import pytz

# 1) 경로
dir_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\전처리"
file1 = os.path.join(dir_path, "KT_형태소라벨_미매칭제거완료.xlsx")
file2 = os.path.join(dir_path, "KT_펨토셀_해킹_videos_20251103_152106.xlsx")

# 2) 로드
df1 = pd.read_excel(file1)
df2 = pd.read_excel(file2)

# 3) 공용 파서 + 타임존 변환
def parse_local(dt_series):
    """KST(naive)로 맞춘 pandas datetime 시리즈 반환"""
    s = pd.to_datetime(dt_series, errors='coerce')  # 다양한 포맷 허용
    # 이미 tz-aware(예: '...Z')면 KST로 변환, 아니면 그대로 사용
    if getattr(s.dt, "tz", None) is not None:
        s = s.dt.tz_convert("Asia/Seoul").dt.tz_localize(None)
    return s

# df1: 보통 KST naive로 존재
df1["datetime"] = parse_local(df1["datetime"])

# df2: YouTube ISO8601(UTC 'Z') → KST로 변환
# utc=True로 강제 파싱 후 tz_convert
published_utc = pd.to_datetime(df2["publishedAt"], errors="coerce", utc=True)
df2["publishedAt_kst"] = published_utc.dt.tz_convert("Asia/Seoul").dt.tz_localize(None)

# 4) 유튜브만 선별
if "channel" not in df1.columns:
    raise KeyError("전처리 데이터(df1)에 'channel' 컬럼이 없습니다.")
df1_youtube = df1[df1["channel"].astype(str).str.strip() == "유튜브"].copy()

# 5) 중복 제거 후 KST 기준으로 정확 일치 병합
if "channelTitle" not in df2.columns:
    raise KeyError("raw 데이터(df2)에 'channelTitle' 컬럼이 없습니다.")

df2_unique = df2[["publishedAt_kst", "channelTitle"]].drop_duplicates(
    subset=["publishedAt_kst"], keep="first"
)

merged_youtube = pd.merge(
    df1_youtube,
    df2_unique,
    how="left",
    left_on="datetime",
    right_on="publishedAt_kst"
).drop(columns=["publishedAt_kst"])

# 6) 유튜브 외는 그대로 유지
df1_non_youtube = df1[df1["channel"].astype(str).str.strip() != "유튜브"].copy()
final_df = pd.concat([df1_non_youtube, merged_youtube], ignore_index=True)

# 7) 저장
save_path = os.path.join(dir_path, "KT_final.xlsx")
final_df.to_excel(save_path, index=False)

# 8) 요약
youtube_count = len(df1_youtube)
filled = merged_youtube["channelTitle"].notna().sum()
rate = (filled / youtube_count * 100) if youtube_count > 0 else 0.0

print(f"저장 완료: {save_path}")
print(f"유튜브 채널 행 수: {youtube_count}")
print(f"정확 일치 매칭 수: {filled} ({rate:.1f}%)")


저장 완료: C:\Users\speec\OneDrive\Desktop\PoC_v2\전처리\KT_final.xlsx
유튜브 채널 행 수: 348
정확 일치 매칭 수: 348 (100.0%)


In [22]:
import pandas as pd
import os
import pytz

# 1) 경로
dir_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\전처리"
file1 = os.path.join(dir_path, "SKT_형태소라벨_미매칭제거완료.xlsx")
file2 = os.path.join(dir_path, "SKT_유심칩_유출_videos_20251103_165204.xlsx")

# 2) 로드
df1 = pd.read_excel(file1)
df2 = pd.read_excel(file2)

# 3) 공용 파서 + 타임존 변환
def parse_local(dt_series):
    """KST(naive)로 맞춘 pandas datetime 시리즈 반환"""
    s = pd.to_datetime(dt_series, errors='coerce')  # 다양한 포맷 허용
    # 이미 tz-aware(예: '...Z')면 KST로 변환, 아니면 그대로 사용
    if getattr(s.dt, "tz", None) is not None:
        s = s.dt.tz_convert("Asia/Seoul").dt.tz_localize(None)
    return s

# df1: 보통 KST naive로 존재
df1["datetime"] = parse_local(df1["datetime"])

# df2: YouTube ISO8601(UTC 'Z') → KST로 변환
# utc=True로 강제 파싱 후 tz_convert
published_utc = pd.to_datetime(df2["publishedAt"], errors="coerce", utc=True)
df2["publishedAt_kst"] = published_utc.dt.tz_convert("Asia/Seoul").dt.tz_localize(None)

# 4) 유튜브만 선별
if "channel" not in df1.columns:
    raise KeyError("전처리 데이터(df1)에 'channel' 컬럼이 없습니다.")
df1_youtube = df1[df1["channel"].astype(str).str.strip() == "유튜브"].copy()

# 5) 중복 제거 후 KST 기준으로 정확 일치 병합
if "channelTitle" not in df2.columns:
    raise KeyError("raw 데이터(df2)에 'channelTitle' 컬럼이 없습니다.")

df2_unique = df2[["publishedAt_kst", "channelTitle"]].drop_duplicates(
    subset=["publishedAt_kst"], keep="first"
)

merged_youtube = pd.merge(
    df1_youtube,
    df2_unique,
    how="left",
    left_on="datetime",
    right_on="publishedAt_kst"
).drop(columns=["publishedAt_kst"])

# 6) 유튜브 외는 그대로 유지
df1_non_youtube = df1[df1["channel"].astype(str).str.strip() != "유튜브"].copy()
final_df = pd.concat([df1_non_youtube, merged_youtube], ignore_index=True)

# 7) 저장
save_path = os.path.join(dir_path, "SKT_final.xlsx")
final_df.to_excel(save_path, index=False)

# 8) 요약
youtube_count = len(df1_youtube)
filled = merged_youtube["channelTitle"].notna().sum()
rate = (filled / youtube_count * 100) if youtube_count > 0 else 0.0

print(f"저장 완료: {save_path}")
print(f"유튜브 채널 행 수: {youtube_count}")
print(f"정확 일치 매칭 수: {filled} ({rate:.1f}%)")


저장 완료: C:\Users\speec\OneDrive\Desktop\PoC_v2\전처리\SKT_final.xlsx
유튜브 채널 행 수: 493
정확 일치 매칭 수: 493 (100.0%)


In [24]:
import pandas as pd
import os
import pytz

# 1) 경로
dir_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\전처리"
file1 = os.path.join(dir_path, "롯데_형태소라벨_미매칭제거완료.xlsx")
file2 = os.path.join(dir_path, "롯데카드_정보유출_videos_20251103_165555.xlsx")

# 2) 로드
df1 = pd.read_excel(file1)
df2 = pd.read_excel(file2)

# 3) 공용 파서 + 타임존 변환
def parse_local(dt_series):
    """KST(naive)로 맞춘 pandas datetime 시리즈 반환"""
    s = pd.to_datetime(dt_series, errors='coerce')  # 다양한 포맷 허용
    # 이미 tz-aware(예: '...Z')면 KST로 변환, 아니면 그대로 사용
    if getattr(s.dt, "tz", None) is not None:
        s = s.dt.tz_convert("Asia/Seoul").dt.tz_localize(None)
    return s

# df1: 보통 KST naive로 존재
df1["datetime"] = parse_local(df1["datetime"])

# df2: YouTube ISO8601(UTC 'Z') → KST로 변환
# utc=True로 강제 파싱 후 tz_convert
published_utc = pd.to_datetime(df2["publishedAt"], errors="coerce", utc=True)
df2["publishedAt_kst"] = published_utc.dt.tz_convert("Asia/Seoul").dt.tz_localize(None)

# 4) 유튜브만 선별
if "channel" not in df1.columns:
    raise KeyError("전처리 데이터(df1)에 'channel' 컬럼이 없습니다.")
df1_youtube = df1[df1["channel"].astype(str).str.strip() == "유튜브"].copy()

# 5) 중복 제거 후 KST 기준으로 정확 일치 병합
if "channelTitle" not in df2.columns:
    raise KeyError("raw 데이터(df2)에 'channelTitle' 컬럼이 없습니다.")

df2_unique = df2[["publishedAt_kst", "channelTitle"]].drop_duplicates(
    subset=["publishedAt_kst"], keep="first"
)

merged_youtube = pd.merge(
    df1_youtube,
    df2_unique,
    how="left",
    left_on="datetime",
    right_on="publishedAt_kst"
).drop(columns=["publishedAt_kst"])

# 6) 유튜브 외는 그대로 유지
df1_non_youtube = df1[df1["channel"].astype(str).str.strip() != "유튜브"].copy()
final_df = pd.concat([df1_non_youtube, merged_youtube], ignore_index=True)

# 7) 저장
save_path = os.path.join(dir_path, "롯데_final.xlsx")
final_df.to_excel(save_path, index=False)

# 8) 요약
youtube_count = len(df1_youtube)
filled = merged_youtube["channelTitle"].notna().sum()
rate = (filled / youtube_count * 100) if youtube_count > 0 else 0.0

print(f"저장 완료: {save_path}")
print(f"유튜브 채널 행 수: {youtube_count}")
print(f"정확 일치 매칭 수: {filled} ({rate:.1f}%)")


저장 완료: C:\Users\speec\OneDrive\Desktop\PoC_v2\전처리\롯데_final.xlsx
유튜브 채널 행 수: 497
정확 일치 매칭 수: 497 (100.0%)
